In [1]:
import polars as pl
import folium
import branca.colormap as cm
from folium.plugins import HeatMap
import pandas as pd
import geopandas as gpd

In [12]:
df = pl.read_csv(
    "brisbane_data_distance.csv/brisbane_data_distance.csv",
    ignore_errors=True,
    null_values=["", "NA", "N310"],
    schema_overrides={
        "origin_stop_lat": pl.Float64,
        "origin_stop_lon": pl.Float64,
        "destination_stop_lat": pl.Float64,
        "destination_stop_lon": pl.Float64,
        "quantity": pl.Int64,
        "distance": pl.Float64,
        "route": pl.Utf8  # allow nulls (e.g., "", NA, N310) in route column
    }
)

df = df.drop("time")
df = df.drop("month")

df.head

<bound method DataFrame.head of shape: (680_382, 17)
┌────────┬─────────────┬───────┬───────────┬───┬─────────────┬─────────────┬────────────┬──────────┐
│        ┆ operator    ┆ route ┆ direction ┆ … ┆ destination ┆ destination ┆ destinatio ┆ distance │
│ ---    ┆ ---         ┆ ---   ┆ ---       ┆   ┆ _stop_type  ┆ _stop_lat   ┆ n_stop_lon ┆ ---      │
│ i64    ┆ str         ┆ str   ┆ str       ┆   ┆ ---         ┆ ---         ┆ ---        ┆ f64      │
│        ┆             ┆       ┆           ┆   ┆ str         ┆ f64         ┆ f64        ┆          │
╞════════╪═════════════╪═══════╪═══════════╪═══╪═════════════╪═════════════╪════════════╪══════════╡
│ 0      ┆ Hornibrook  ┆ 680   ┆ Inbound   ┆ … ┆ Bus         ┆ -27.383079  ┆ 153.030287 ┆ 2.983873 │
│        ┆ Bus Lines   ┆       ┆           ┆   ┆             ┆             ┆            ┆          │
│ 1      ┆ Hornibrook  ┆ 680   ┆ Inbound   ┆ … ┆ Virtual     ┆ -27.385876  ┆ 153.031644 ┆ 3.321307 │
│        ┆ Bus Lines   ┆       ┆      

In [21]:
filtered_df = df.filter(
    (pl.col("origin_stop_name") == "Lone Pine Koala Sanctuary at Lone Pine, stop 42") &
    (pl.col("destination_stop_name") == "Adelaide Street Stop 22 near City Hall")
)

result = (
    filtered_df
    .group_by(["origin_stop_name", "destination_stop_name"])
    .agg([
        pl.sum("quantity").alias("total_quantity")
    ])
)
result

origin_stop_name,destination_stop_name,total_quantity
str,str,i64
"""Lone Pine Koala Sanctuary at L…","""Adelaide Street Stop 22 near C…",302


In [46]:
grouped_df = df.group_by([
    "origin_stop", "destination_stop"
]).agg([
    pl.first("operator").alias("operator"),
    pl.first("route").alias("route"),
    pl.first("direction").alias("direction"),
    pl.first("ticket_type").alias("ticket_type"),
    pl.first("origin_stop_name").alias("origin_stop_name"),
    pl.first("origin_stop_type").alias("origin_stop_type"),
    pl.first("origin_stop_lat").alias("origin_stop_lat"),
    pl.first("origin_stop_lon").alias("origin_stop_lon"),
    pl.first("destination_stop_name").alias("destination_stop_name"),
    pl.first("destination_stop_type").alias("destination_stop_type"),
    pl.first("destination_stop_lat").alias("destination_stop_lat"),
    pl.first("destination_stop_lon").alias("destination_stop_lon"),
    pl.sum("quantity").alias("quantity"),
    pl.first("distance").alias("distance")
])
grouped_df = grouped_df.sort("distance")
grouped_df


origin_stop,destination_stop,operator,route,direction,ticket_type,origin_stop_name,origin_stop_type,origin_stop_lat,origin_stop_lon,destination_stop_name,destination_stop_type,destination_stop_lat,destination_stop_lon,quantity,distance
i64,i64,str,i64,str,str,str,str,f64,f64,str,str,f64,f64,i64,f64
1951,1951,"""Transport for Brisbane""",460,"""Inbound""","""go card""","""Coronation Dr at Wesley Hospit…","""Bus""",-27.480073,152.998002,"""Coronation Dr at Wesley Hospit…","""Bus""",-27.480073,152.998002,20,0.0
10979,10979,"""Transport for Brisbane""",460,"""Inbound""","""go card""","""Forest Lake Bvd at Forest Lake…","""Bus""",-27.610239,152.957668,"""Forest Lake Bvd at Forest Lake…","""Bus""",-27.610239,152.957668,35,0.0
1747,1747,"""Transport for Brisbane""",460,"""Inbound""","""go card""","""Moggill Rd at BBC, stop 16""","""Bus""",-27.489381,152.987746,"""Moggill Rd at BBC, stop 16""","""Bus""",-27.489381,152.987746,39,0.0
5271,5271,"""Transport for Brisbane""",460,"""Inbound""","""go card""","""Mount Ommaney Shopping Centre""","""Bus""",-27.54879,152.937899,"""Mount Ommaney Shopping Centre""","""Bus""",-27.54879,152.937899,249,0.0
5302,5302,"""Transport for Brisbane""",460,"""Inbound""","""go card""","""Dandenong Rd near Horizon Dr""","""Bus""",-27.557656,152.933609,"""Dandenong Rd near Horizon Dr""","""Bus""",-27.557656,152.933609,49,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
11255,929,"""Transport for Brisbane""",460,"""Inbound""","""go card""","""Parkwood Dr near Acacia St""","""Bus""",-27.636805,152.990261,"""Queen Street station""","""Virtual""",-27.470269,153.024518,425,18.761822
11259,929,"""Transport for Brisbane""",460,"""Inbound""","""go card""","""Parkwood Dr near Watergum St""","""Bus""",-27.636607,152.988942,"""Queen Street station""","""Virtual""",-27.470269,153.024518,2147,18.764198
11253,10792,"""Transport for Brisbane""",460,"""Inbound""","""go card""","""Parkwood Dr near Coolibah St""","""Bus""",-27.635364,152.991521,"""Roma Street busway, platform 2""","""Busway""",-27.465942,153.0192,90,18.972026


In [5]:

# Step 1: Convert Polars DataFrame to Pandas
trips_df = grouped_df.sort("distance").to_pandas().reset_index(drop=True)

# Step 2: Re-link origin to previous destination to simulate chained path
trips_df["origin_stop"] = trips_df["destination_stop"].shift(1)
trips_df["origin_stop_lat"] = trips_df["destination_stop_lat"].shift(1)
trips_df["origin_stop_lon"] = trips_df["destination_stop_lon"].shift(1)
trips_df["origin_stop_name"] = trips_df["destination_stop_name"].shift(1)

# Step 3: Drop rows where origin is now NaN
trips_df = trips_df.dropna(subset=["origin_stop"])

threshold = trips_df["quantity"].quantile(0.95)  # top 20%
trips_df = trips_df[trips_df["quantity"] >= threshold]

# Step 4: Set up colormap based on quantity
min_q = trips_df["quantity"].min()
max_q = trips_df["quantity"].max()
colormap = cm.LinearColormap(colors=["blue", "lime", "red"], vmin=min_q, vmax=max_q)

# Step 5: Normalize quantity column to range 0–1
trips_df["normalized_quantity"] = (trips_df["quantity"] - min_q) / (max_q - min_q)

# Step 6: Create map
m = folium.Map(
    location=[trips_df["origin_stop_lat"].mean(), trips_df["origin_stop_lon"].mean()],
    zoom_start=13,
    tiles="CartoDB positron"
)

# Step 7: Draw PolyLines with color and opacity based on quantity
for _, row in trips_df.iterrows():
    folium.PolyLine(
        locations=[
            [row["origin_stop_lat"], row["origin_stop_lon"]],
            [row["destination_stop_lat"], row["destination_stop_lon"]],
        ],
        color="blue",
        weight=2 + row["normalized_quantity"] * 6,
        opacity=max(row["normalized_quantity"], 0.1),  # ensure low values are still visible
        tooltip=(
            f"{row['origin_stop_name']} → {row['destination_stop_name']}<br>"
            f"{row['quantity']} passengers"
        )
    ).add_to(m)

# Step 8: Add colormap legend
colormap.caption = "Passenger Quantity"
colormap.add_to(m)

# Step 9: Display map in Jupyter
m

In [6]:
# If not already, convert grouped_df to pandas
trips_df = grouped_df.to_pandas()

# Calculate midpoint between origin and destination for each trip
trips_df["mid_lat"] = (trips_df["origin_stop_lat"] + trips_df["destination_stop_lat"]) / 2
trips_df["mid_lon"] = (trips_df["origin_stop_lon"] + trips_df["destination_stop_lon"]) / 2

# Drop nulls just in case
midpoints = trips_df[["mid_lat", "mid_lon", "quantity"]].dropna()

# Convert to list of [lat, lon, weight]
heat_data = midpoints.values.tolist()

In [7]:
# Create base map
m = folium.Map(
    location=[midpoints["mid_lat"].mean(), midpoints["mid_lon"].mean()],
    zoom_start=13,
    tiles="CartoDB positron"
)

# Add the heatmap layer
HeatMap(heat_data, radius=17, blur=15, max_zoom=1).add_to(m)

# Display the map
m


Suburb mapping

In [5]:
import polars as pl
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Load your original dataset
df = pl.read_csv(
    "brisbane_data_distance.csv/brisbane_data_distance.csv",
    ignore_errors=True,
    null_values=["", "NA", "N310"],
    schema_overrides={
        "origin_stop_lat": pl.Float64,
        "origin_stop_lon": pl.Float64,
        "destination_stop_lat": pl.Float64,
        "destination_stop_lon": pl.Float64,
        "quantity": pl.Int64,
        "distance": pl.Float64
    }
)

# Drop unnecessary and invalid stop types
df = df.drop("time").drop("month")

# Load suburb boundaries
suburbs_gdf = gpd.read_file(
    r"QSC_Extracted_Data_20250416_015956289270-10580/QSC_Extracted_Data_20250416_015956289270-10580/data.gdb",
    layer="Locality_Boundaries"
)
suburbs_gdf = suburbs_gdf.to_crs(epsg=4326)

# Chunked origin + destination suburb tagging
chunk_size = 250_000
origin_chunks = []
destination_chunks = []

for i in range(0, df.height, chunk_size):
    chunk_df = df.slice(i, chunk_size).to_pandas()

    # Origin suburb
    chunk_df["geometry"] = chunk_df.apply(
        lambda row: Point(row["origin_stop_lon"], row["origin_stop_lat"]), axis=1
    )
    origin_gdf = gpd.GeoDataFrame(chunk_df, geometry="geometry", crs="EPSG:4326")
    origin_joined = gpd.sjoin(origin_gdf, suburbs_gdf, how="left", predicate="within")
    origin_chunk = origin_joined[["adminareaname"]].rename(columns={"adminareaname": "origin_suburb"})
    origin_chunks.append(origin_chunk)

    # Destination suburb
    chunk_df["geometry"] = chunk_df.apply(
        lambda row: Point(row["destination_stop_lon"], row["destination_stop_lat"]), axis=1
    )
    destination_gdf = gpd.GeoDataFrame(chunk_df, geometry="geometry", crs="EPSG:4326")
    destination_joined = gpd.sjoin(destination_gdf, suburbs_gdf, how="left", predicate="within")
    destination_chunk = destination_joined[["adminareaname"]].rename(columns={"adminareaname": "destination_suburb"})
    destination_chunks.append(destination_chunk)

# Combine all suburb results
origin_suburbs = pd.concat(origin_chunks, ignore_index=True)
destination_suburbs = pd.concat(destination_chunks, ignore_index=True)

# Attach both to main DataFrame
df_pd = df.to_pandas()
df_pd["origin_suburb"] = origin_suburbs["origin_suburb"].values
df_pd["destination_suburb"] = destination_suburbs["destination_suburb"].values

# Optional: Convert back to Polars
df_final = pl.from_pandas(df_pd)


In [6]:
df_final

column_0,operator,route,direction,ticket_type,origin_stop,destination_stop,quantity,origin_stop_name,origin_stop_type,origin_stop_lat,origin_stop_lon,destination_stop_name,destination_stop_type,destination_stop_lat,destination_stop_lon,distance,origin_suburb,destination_suburb
i64,str,f64,str,str,f64,f64,f64,str,str,f64,f64,str,str,f64,f64,f64,str,str
0,"""Hornibrook Bus Lines""",680.0,"""Inbound""","""go card""",10064.0,3940.0,8.0,"""Gympie Rd at Brickfield Street…","""Bus""",-27.358884,153.017047,"""Gympie Rd at The Gardens, stop…","""Bus""",-27.383079,153.030287,2.983873,"""ASPLEY, BRISBANE CITY""","""CHERMSIDE, BRISBANE CITY"""
1,"""Hornibrook Bus Lines""",680.0,"""Inbound""","""go card""",10064.0,3980.0,7.0,"""Gympie Rd at Brickfield Street…","""Bus""",-27.358884,153.017047,"""Chermside Shopping Centre""","""Virtual""",-27.385876,153.031644,3.321307,"""ASPLEY, BRISBANE CITY""","""CHERMSIDE, BRISBANE CITY"""
2,"""Hornibrook Bus Lines""",680.0,"""Inbound""","""go card""",10067.0,3940.0,9.0,"""Gympie Rd at Aspley""","""Bus""",-27.361871,153.017315,"""Gympie Rd at The Gardens, stop…","""Bus""",-27.383079,153.030287,2.677589,"""ASPLEY, BRISBANE CITY""","""CHERMSIDE, BRISBANE CITY"""
3,"""Hornibrook Bus Lines""",680.0,"""Inbound""","""go card""",10067.0,3980.0,25.0,"""Gympie Rd at Aspley""","""Bus""",-27.361871,153.017315,"""Chermside Shopping Centre""","""Virtual""",-27.385876,153.031644,3.014096,"""ASPLEY, BRISBANE CITY""","""CHERMSIDE, BRISBANE CITY"""
4,"""Hornibrook Bus Lines""",680.0,"""Inbound""","""go card""",10391.0,10067.0,1.0,"""Gympie Rd near Bald Hills Rd""","""Bus""",-27.321863,153.009095,"""Gympie Rd at Aspley""","""Bus""",-27.361871,153.017315,4.507245,"""BALD HILLS, BRISBANE CITY""","""ASPLEY, BRISBANE CITY"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
680377,"""Transport for Brisbane""",836.0,"""Outbound""","""go card""",2328.0,2321.0,1.0,"""Bapaume Rd at Marshall Road, s…","""Bus""",-27.527061,153.056103,"""Prior St at Windmill Street, s…","""Bus""",-27.531319,153.04246,1.427946,"""TARRAGINDI, BRISBANE CITY""","""TARRAGINDI, BRISBANE CITY"""
680378,"""Transport for Brisbane""",836.0,"""Outbound""","""go card""",2328.0,3032.0,1.0,"""Bapaume Rd at Marshall Road, s…","""Bus""",-27.527061,153.056103,"""Andrew Ave at Tarragindi Bowls…","""Bus""",-27.52556,153.037823,1.8135,"""TARRAGINDI, BRISBANE CITY""","""TARRAGINDI, BRISBANE CITY"""
680379,"""Transport for Brisbane""",836.0,"""Outbound""","""go card""",2328.0,3037.0,5.0,"""Bapaume Rd at Marshall Road, s…","""Bus""",-27.527061,153.056103,"""Bramston St near Andrew Ave, s…","""Bus""",-27.529269,153.041033,1.508692,"""TARRAGINDI, BRISBANE CITY""","""TARRAGINDI, BRISBANE CITY"""


In [7]:
import folium
# Filter the suburb GeoDataFrame to just "Forest Lake"
forest_lake_poly = suburbs_gdf[suburbs_gdf["adminareaname"] == "FOREST LAKE, BRISBANE CITY"]

# Get the centroid to center the map
center = forest_lake_poly.geometry.centroid.iloc[0]
map_center = [center.y, center.x]

# Create a folium map centered on Forest Lake
m = folium.Map(location=map_center, zoom_start=13, tiles="CartoDB positron")

# Add the Forest Lake polygon
folium.GeoJson(
    forest_lake_poly,
    name="Forest Lake",
    style_function=lambda x: {
        "fillColor": "orange",
        "color": "red",
        "weight": 2,
        "fillOpacity": 0.4
    },
    tooltip=folium.GeoJsonTooltip(fields=["adminareaname"], aliases=["Suburb"])
).add_to(m)

m

C:\Users\Admin\AppData\Local\Temp\ipykernel_1480\3004935810.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = forest_lake_poly.geometry.centroid.iloc[0]


In [8]:
# Step 1: Filter for Forest Lake origin before grouping
filtered_df = df_final.filter(
    (pl.col("origin_suburb") == "FOREST LAKE, BRISBANE CITY") &
    (pl.col("direction") == "Inbound")
)
# Step 2: Group and aggregate
grouped_df = filtered_df.group_by([
    "origin_stop", "destination_stop"
]).agg([
    pl.first("operator").alias("operator"),
    pl.first("route").alias("route"),
    pl.first("direction").alias("direction"),
    pl.first("ticket_type").alias("ticket_type"),
    pl.first("origin_stop_name").alias("origin_stop_name"),
    pl.first("origin_stop_type").alias("origin_stop_type"),
    pl.first("origin_stop_lat").alias("origin_stop_lat"),
    pl.first("origin_stop_lon").alias("origin_stop_lon"),
    pl.first("destination_stop_name").alias("destination_stop_name"),
    pl.first("destination_stop_type").alias("destination_stop_type"),
    pl.first("destination_stop_lat").alias("destination_stop_lat"),
    pl.first("destination_stop_lon").alias("destination_stop_lon"),
    pl.sum("quantity").alias("quantity"),
    pl.first("distance").alias("distance")
])

# Step 3: Sort by distance
grouped_df = grouped_df.sort("distance")
grouped_df

origin_stop,destination_stop,operator,route,direction,ticket_type,origin_stop_name,origin_stop_type,origin_stop_lat,origin_stop_lon,destination_stop_name,destination_stop_type,destination_stop_lat,destination_stop_lon,quantity,distance
f64,f64,str,f64,str,str,str,str,f64,f64,str,str,f64,f64,f64,f64
51702.0,51702.0,"""Transport for Brisbane""",770.0,"""Inbound""","""go card""","""College Ave near Oxley Place""","""Bus""",-27.612308,152.972581,"""College Ave near Oxley Place""","""Bus""",-27.612308,152.972581,2.0,0.0
10974.0,10974.0,"""Transport for Brisbane""",100.0,"""Inbound""","""go card""","""Woodland Ave near Laricina Cct""","""Bus""",-27.61093,152.971616,"""Woodland Ave near Laricina Cct""","""Bus""",-27.61093,152.971616,9.0,0.0
10969.0,10969.0,"""Transport for Brisbane""",100.0,"""Inbound""","""go card""","""Forest Lake Shops""","""Bus""",-27.623729,152.96993,"""Forest Lake Shops""","""Bus""",-27.623729,152.96993,92.0,0.0
1971.0,1971.0,"""Transport for Brisbane""",779.0,"""Inbound""","""go card""","""High St at Forest Lake High Sc…","""Virtual""",-27.624645,152.971248,"""High St at Forest Lake High Sc…","""Virtual""",-27.624645,152.971248,1.0,0.0
10803.0,10803.0,"""Transport for Brisbane""",100.0,"""Inbound""","""go card""","""Grand Ave East near Ascot Ave""","""Bus""",-27.627464,152.969834,"""Grand Ave East near Ascot Ave""","""Bus""",-27.627464,152.969834,9.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
10810.0,148.0,"""Transport for Brisbane""",118.0,"""Inbound""","""go card""","""Grand Ave East near Ascot Ave""","""Bus""",-27.627456,152.968325,"""Creek Street Stop 148 at River…","""Bus""",-27.467314,153.028827,92.0,18.724827
4396.0,148.0,"""Transport for Brisbane""",118.0,"""Inbound""","""go card""","""Grand Ave at Grand Avenue Scho…","""Bus""",-27.625229,152.960397,"""Creek Street Stop 148 at River…","""Bus""",-27.467314,153.028827,22.0,18.758841
10719.0,148.0,"""Transport for Brisbane""",118.0,"""Inbound""","""go card""","""Grand Ave at Grand Avenue Lake""","""Bus""",-27.626995,152.963535,"""Creek Street Stop 148 at River…","""Bus""",-27.467314,153.028827,24.0,18.833137


In [9]:
import pandas as pd
import folium

trips_pd = grouped_df.to_pandas()


In [10]:
min_q = trips_pd["quantity"].min()
max_q = trips_pd["quantity"].max()
trips_pd["normalized_quantity"] = (trips_pd["quantity"] - min_q) / (max_q - min_q)

# Use the first origin coordinate as the center
lat_center = trips_pd["origin_stop_lat"].iloc[0]
lon_center = trips_pd["origin_stop_lon"].iloc[0]

m = folium.Map(location=[lat_center, lon_center], zoom_start=13, tiles="CartoDB positron")



In [1]:
import folium
threshold = trips_pd["quantity"]

top_trips = trips_pd[trips_pd["quantity"] >= threshold]

# Get Forest Lake boundary from suburbs_gdf
forest_lake_poly = suburbs_gdf[suburbs_gdf["adminareaname"] == "FOREST LAKE, BRISBANE CITY"]

# Calculate centroid for map center
center = forest_lake_poly.geometry.centroid.iloc[0]
center_lat = center.y
center_lon = center.x

# Create folium map
m = folium.Map(location=[center_lat, center_lon], zoom_start=13, tiles="CartoDB positron")

# 🟥 Add Forest Lake boundary
folium.GeoJson(
    forest_lake_poly,
    name="Forest Lake",
    style_function=lambda x: {
        "fillColor": "orange",
        "color": "red",
        "weight": 3,
        "fillOpacity": 0.2
    },
    tooltip=folium.GeoJsonTooltip(fields=["adminareaname"], aliases=["Suburb"])
).add_to(m)

# ➖ Add trip lines
for _, row in top_trips.iterrows():
    folium.PolyLine(
        locations=[
            [row["origin_stop_lat"], row["origin_stop_lon"]],
            [row["destination_stop_lat"], row["destination_stop_lon"]],
        ],
        color="blue",
        weight=2 + row["normalized_quantity"] * 6,
        opacity=max(row["normalized_quantity"], 0.1),
        tooltip=f"{row['quantity']} passengers to {row['destination_stop_name']}"
    ).add_to(m)

m


NameError: name 'trips_pd' is not defined

In [3]:
import pandas as pd
import ast

# Load CSV
df = pd.read_csv("trip_summary.csv")

df.head

<bound method NDFrame.head of     origin_suburb                                top_1_percent_trips
0    ACACIA RIDGE  {'INALA': [('Bus', '110', 'Outbound', 'Inala P...
1          ALBION  {'FORTITUDE VALLEY': [('Rail', 'Rail', 'Rail',...
2        ALDERLEY  {'BOWEN HILLS': [('Rail', 'Rail', 'Rail', 'Bow...
3        ALGESTER  {'SUNNYBANK HILLS': [('Bus', '130', 'Inbound',...
4        ANNERLEY  {'ANNERLEY': [('Bus', '100', 'Outbound', 'Ipsw...
..            ...                                                ...
170        WYNNUM  {'BRISBANE CITY': [('Rail', 'Rail', 'Rail', 'C...
171   WYNNUM WEST  {'WYNNUM': [('Rail', 'Rail', 'Rail', 'Wynnum C...
172  YEERONGPILLY  {'SOUTH BRISBANE': [('Rail', 'Rail', 'Rail', '...
173       YERONGA  {'FAIRFIELD': [('Bus', '109', 'Inbound', 'Fair...
174      ZILLMERE  {'CHERMSIDE': [('Bus', '330', 'Inbound', 'Gymp...

[175 rows x 2 columns]>